# Olive API for Python Tutorial 

This tutorial explains how to use the Olive API for Python to access the Olive server.  This is a new feature in Olive 5.0 and can be used to write your own Python scripts.

This interface is client-server oriented.  You are writing a client, which will send requests to the server.  The basic flow is to:
1. Create a client object.
2. Connect to the server.
3. Send one or more requests the server.
4. Disconnect from the server.

Important notes about this library:
* Olive 5.0 uses Python 3.  The interface is tested with Python 3.8.1 on Windows and 3.6.7 on Linux, but should work with all 3.x versions on all platforms.
* The python directory where this source code is located must be in your PYTHONHOME environment variable.
* The Olive runtime must also be in your PYTHONHOME.
* There are no timeouts, so if your request "hangs" forever, that usually means that the server is not running or is not running on the machine or port you have connected to.
* Most examples do not handle server side exceptions. See the Error Handling example for one that does.

List of Examples

- Quick Start Examples
    - First Example: Connect and Run A Frame Analysis Using the SAD Plugin
    - Second Example: Enroll and Run A Region Analysis Using the SDD Plugin
- Multi Threaded / Async Example
- Specific Examples

# Quick Start Examples

These examples are designed to get you started quickly.

## First Example: Connect and Run A Frame Analysis Using the SAD Plugin
SAD stands for Sound Automated Detection <br>
The simplest example is to connected to the server, have it do
a speach detection, using the SAD plugin, print what it found,
and then disconnect from the server.import olive.oliveclient as oc

In [1]:
import olivepy.api.oliveclient as oc

The argument is a name, and can be anything. <br>
If you want to change the port or run the server on a different machine, see below.

In [2]:
client = oc.OliveClient('testcient')
client.connect()

Client.connect does not return anything, so we check success like this:

In [3]:
if not client.is_connected():
    print('Connection failed.  Is the server up and running?')
    quit(1)

In this example we get frame data from a SAD request. <br>
The first argument is the plugin name. <br>
The second argument is the domain name within that plugin. <br>
The third agument is the path to the audio file, relative to the server. <br>
*Note: This path is relative to where the server is running, not the client.* <br>

In [ ]:
frames = client.analyze_frames('sad-dnn-v6b-py3', 'multi-v1', 'doc\TwoSpeakers.wav')

The analyze_frames function returns an array of numbers.

In [ ]:
num_frames = len(frames)
print('Got results for %d frames.' % num_frames)
if num_frames > 4:
    print('First two results are %f %f and last two are %f %f.' % (frames[0], frames[1], frames[-2], frames[-1]))
elif num_frames > 0:
    print('First result is %f and last is %f.' % (frames[0], frames[-1]))

Or you can iterate through it.

In [ ]:
sum = 0.0
for frame in frames:
    sum += frame
print('sum of frame values %f.' % sum)

And if we are not going to do any more analysis, we disconnect from the server.
This function does not return anything.

In [ ]:
client.disconnect()

### First Example: All The Code

In [ ]:
import olivepy.api.oliveclient as oc
client = oc.OliveClient('testcient')
client.connect()
if not client.is_connected():
    print('Connection failed.  Is the server up and running?')
    quit(1)
num_frames = len(frames)
print('Got results for %d frames.' % num_frames)
if num_frames > 4:
    print('First two results are %2.4f %2.4f, last two are %2.4f %2.4f.' % (frames[0], frames[1], frames[-2], frames[-1]))
elif num_frames > 0:
    print('First result is %2.4f and last is %2.4f.' % (frames[0], frames[-1]
client.disconnect()                                                 
                                                  

## Connection Details

The following code opens the client, which will be used for the rst of the examples here. <br>
The three extra arguments to OliveClient specify the machine running the server, the port number and a timeout. <br>
*Note: OliveClient creation takes an optional timeout argument, but it is not current used.* <br>
*Note: Connect takes an optional argument but it is not currently supported.* <br>

In [ ]:
client = oc.OliveClient('testcient', 'localhost', 5588)
client.connect()
if not client.is_connected():
    print('Connection failed.  Is the server up and running?')
    quit(1)
# In the next line, you must set the plugin, domain, and wav file to the 
# right names for your environment.
frames = client.analyze_frames('sad-dnn-v6b-py3', 'multi-v1', 'TwoSpeakers.wav')    

## Second Example: Enroll and Run A Region Analysis Using the SDD Plugin

The first argument is the plugin name. <br>
The second argument is the domain name within that plugin. <br>
The third argument is the name of the sound you are enrolling. <br>
The forth agument is the path to the audio file, relative to the server. <br>
*Note: This path is relative to where the server is running, not the client.* <br>

In [ ]:
who = 'joshua'
success = client.enroll('sdd-sbcEmbed-v1b-py3', 'telClosetalk-v1', who, 'doc\joshua2.wav')
if success:
    print('Successfully enrolled {} in {} {}'.format(who, 'sdd-sbcEmbed-v1b-py3', 'telClosetalk-v1'))
else:
    print('Enrollment of {} failed.'.format(who))
    quit(1)

The analyze_regions function does the region analysis.
The first argument is the plugin name. <br>
The second argument is the domain name within that plugin. <br>
The third agument is the path to the audio file, relative to the server. <br>
*Note: This path is relative to where the server is running, not the client.* <br>

In [ ]:
regions = client.analyze_regions('sdd-sbcEmbed-v1b-py3', 'telClosetalk-v1', 'doc\TwoSpeakers.wav')

The analyze_regions function returns a list of tuples, each of which has a class_id, start_t, end_t, and a score.

In [ ]:
for region in regions:
    print('{} {}-{} {}'.format(region.class_id, region.start_t, region.end_t, region.score))

## Third Example: Run A Global Analysis Using the LID Plugin

The first argument is the plugin name. <br>
The second argument is the domain name within that plugin. <br>
The third agument is the path to the audio file, relative to the server. <br>
*Note: This path is relative to where the server is running, not the client.* <br>

In [ ]:
results = client.analyze_global('lid-embedplda-v1b-py3', 'multi-v1', 'doc\TwoSpeakers.wav')

The analyze_global function returns a list of tuples, each of which has a class_id and a score. 

The class_id text returned depends on the plugin used.  In this example, we are using the Language IDentification (LID) plugin, so the class_ids are three letter language codes (ISO 639-2):

In [ ]:
rus = eng = "n/a"
for result in results:
    if result.class_id == 'eng':
        eng = result.score
    if result.class_id == 'rus':
        rus = result.score
print('Got {} results.  Eng is {} and rus is {}.'.format(len(results), eng, rus))  

## Error Handling

Most of the time you do not need to catch the ExceptionFromServer exceptions specifically when calling the server, because whatever exception handling you are doing "above" the call will handle these errors along with the rest.  However, all of the analyze and enroll function calls will raise an ExceptionFromServer exception if any exception occured on the server side while doing the work requested by the call.

In [ ]:
try:
    results = client.analyze_global('lid-nothere', 'multi-v1', 'doc\TwoSpeakers.wav')
    if len(results) != 19:
        # We were expecting 19 language results, but got some other number...
        raise Exception('Got wrong number of language results.  Got %d, expected 19.' % len(results))
except oc.ExceptionFromServer:
    print('Got an exception from the server while doing this analysis: '+ str(ExceptionFromServer))

The most common causes of exceptions from the server from the server are:
* Mispelling the name of the audio file or using a relative path which is not relative from the server.  (Absolute paths are often easier to use.)
* Mispelling the plugin or domain names.
* Trying to use a plugin that the server had not loaded.

Remember that if you try to connect to a server that is not running, or not running where you expect it to be, you will not get an exception: your script will just wait forever.

## Getting More Information on Errors

There are two extra pieces of information available for all calls to the server: info and fullobj.<br>
**Info** is a string that the server may return which contains extra information about the call.  It might contain more information on both succeses and failures, but the server may not provide info for all calls.<br>
**fullobj** is the raw protobuf Python object returned from the remote call to the server.  It may contain information useful for troubleshooting.<br>

Below is an example of use, in an enroll call fails:

In [ ]:
success = client.enroll('sid-embed-v5-py3', 'multilang-v1', 'joshua', 'non-existant-file')
if not success:
    info = self.client.get_info()
    print('Enrollment failed because: '+info)
    fullobj = self.client.get_fullobj()
    print(str(fullobj))
else:
    print('Enrollment was successful.')

## Sending Data To The Server

In the previous examples we have send the path of the sound file on the server to the server.
However, if the sound file is on the client, we can send it over the network like this:

In [ ]:
with open('..\..\..\..\..\..\doc\TwoSpeakers.wav', "rb") as wav_file:
    audio_buffer = wav_file.read()
regions = client.analyze_regions('qbe-tdnn-v7-py3', 'multi-v1',
                                 audio_buffer, self.client.AUDIO_SERIALIZED)

print('Results for qbe-tdnn-v7-py3, multi-v1, serialized doc\TwoSpeakers.wav')
for region in regions:
    print('{} {}-{} {}'.format(region.class_id, region.start_t, region.end_t, region.score))    

## The End
Since we connected to a server, we now must disconnect.

In [ ]:
client.disconnect()

# Multi Threaded / Async Example

In this section you will be walked through creating a program which does many Olive enrollments at the same time.  This same idea can be used for doing many analysis runs at once.

**The key point is this: A different OliveClient object should be used for each thread.** <br>
Just as if you are doing multiprocessing, when each process uses its own Olive Client object.  The same is true for threads.  You can think of an OliveClient as a stream of requests to the Olive server; after each request it will wait for a reponse.  If you want to send many requests at the same time, you need many OliveClient objects to do it.

## Enroll A Class In Many Plugins At Once (with Different Threads For Each Plugin)

We start with the imports for multithreading and using the Olive client:

In [ ]:
import threading
import olivepy.api.oliveclient as oc

This function is going to do the "heavy lifting".  Each thread will run this function to create a client, connect to the server, and enroll in one plugin/domain pair.  The important part is that it creates and uses it's own OliveClient object.  This ensures that it will wait for its own enrollment to finish before continuing.  

In [ ]:
def do_global_analyze(plugin, domain, enroll, wav):
    thread_name = threading.current_thread().getName()
    client = oc.OliveClient(thread_name+'cient')
    client.connect()
    print('On {} enrolling {} into {}#{} with {}'.format(thread_name, enroll, plugin, domain, wav))
    results = client.enroll(plugin, domain, enroll, wav)
    print('On {} result from {}-{} on {}: {}'.format(thread_name, plugin, domain, wav, results))
    client.disconnect()

This function is only needed for multithreaded programs.  For those programs, you must call this function from the main thread, so it can properly set up your signals so that control-C will exit your program.

In [ ]:
oc.OliveClient.setup_multithreading() 

The global_plugins list contains the data used by each thread, and the rest of the program just creates the threads and athen waits for them to complete.

In [ ]:
threads = []
work_tasks = [ ['sid-embed-v5-py3', 'multilang-v1', 'doc\joshua2.wav'],
               ['sid-embed-v5-py3', 'multilang-v1', 'doc\joshua2.wav'], 
               ['sdd-sbcEmbed-v1b-py3', 'telClosetalk-v1', 'doc\joshua2.wav'],
               ['tpd-embed-v1-py3', 'eng-cts-v1', '..\testdata\TaadA_1min.wav'],
               ['tpd-embed-v1-py3', 'rus-cts-v1', '..\testdata\TaadA_1min.wav'],
               ['qbe-tdnn-v7-py3', 'digPtt-v1', 'doc\short1.wav'],
               ['qbe-tdnn-v7-py3', 'multi-v1', 'doc\short1.wav'], ]

for work_task in work_tasks:
    t = threading.Thread(target=do_global_analyze, args=(work_task[0], work_task[1], 'joshua', work_task[2]))
    threads.append(t)
    t.start()

for num in range(len(work_tasks)):
    threads[num].join()

## All The Code Together
Below is the entire program:

In [ ]:
import threading

import olivepy.api.oliveclient as oc

threads = []
work_tasks = [ ['sid-embed-v5-py3', 'multilang-v1', 'doc\joshua2.wav'],
               ['sid-embed-v5-py3', 'multilang-v1', 'doc\joshua2.wav'], 
               ['sdd-sbcEmbed-v1b-py3', 'telClosetalk-v1', 'doc\joshua2.wav'],
               ['tpd-embed-v1-py3', 'eng-cts-v1', '..\testdata\TaadA_1min.wav'],
               ['tpd-embed-v1-py3', 'rus-cts-v1', '..\testdata\TaadA_1min.wav'],
               ['qbe-tdnn-v7-py3', 'digPtt-v1', 'doc\short1.wav'],
               ['qbe-tdnn-v7-py3', 'multi-v1', 'doc\short1.wav'], ]

def do_global_analyze(plugin, domain, enroll, wav):
    thread_name = threading.current_thread().getName()
    client = oc.OliveClient(thread_name+'cient')
    client.connect()
    print('On {} enrolling {} into {}#{} with {}'.format(thread_name, enroll, plugin, domain, wav))
    results = client.enroll(plugin, domain, enroll, wav)
    print('On {} result from {}-{} on {}: {}'.format(thread_name, plugin, domain, wav, results))
    client.disconnect()
    
oc.OliveClient.setup_multithreading()    

for work_task in work_tasks:
    t = threading.Thread(target=do_global_analyze, args=(work_task[0], work_task[1], 'joshua', work_task[2]))
    threads.append(t)
    t.start()

for num in range(len(work_tasks)):
    threads[num].join()

# Specific Examples

## Filter For Languages in LID

TBD

In [ ]:
wanted_classes = [ 'eng', 'rus']
filtered_results = {}
try:
    results = client.analyze_global('lid-embedplda-v1b-py3', 'multi-v1', 'doc\TwoSpeakers.wav')
    for possible in results:
        if possible.class_id in wanted_classes:
            filtered_results.add(possible.class_id, possible.score)
except oc.ExceptionFromServer:
    print('Got an exception from the server while doing this analysis: '+ str(ExceptionFromServer))
    
print filtered_results    

## Filter For Languages by minimum score

TBD

In [ ]:
minimum_level = 5
filtered_results = {}
try:
    results = client.analyze_global('lid-embedplda-v1b-py3', 'multi-v1', 'doc\TwoSpeakers.wav')
    for possible in results:
        if possible.score > minimum_level:
            filtered_results.add(possible.class_id, possible.score)
except oc.ExceptionFromServer:
    print('Got an exception from the server while doing this analysis: '+ str(ExceptionFromServer))
    
print filtered_results